In [3]:
import os
import pandas as pd
import numpy as np
from skimage import filters

In [8]:
pose_threshold = 30 # Euclidian distance for "no significant movement"
window_size = 6 # Frames for non-minimum suppression (24 per second), 
                # half of actual window size because both sides are checked
duplicate_threshold= 150 # prevent consecutive duplicate poses
in_dir = "../data/2_angles"
out_dir = "../data/3_poses"

In [5]:
# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# Function to calculate Euclidean distances between consecutive rows
def calculate_distances(df):
    distances = []
    for i in range(1, len(df)):
        distance = euclidean_distance(df.iloc[i-1], df.iloc[i])
        distances.append(distance)
    return distances

def non_minimum_suppression(df, distances):
    outliers = []
    for i in range(0, len(distances)-2):
        if distances[i] < pose_threshold and distances[i+1] < pose_threshold:
            min_in_window = min(distances[max(0, i-window_size):min(len(distances), i+window_size+1)])
            if distances[i] == min_in_window:
                if len(outliers) == 0 or euclidean_distance(df.iloc[outliers[-1]], df.iloc[i]) > duplicate_threshold:
                    outliers.append(i)
                elif distances[outliers[-1]] + distances[outliers[-1]+1] > distances[i] + distances[i+1]:
                    outliers[-1] = i
    return outliers

In [50]:
filepath = "../data/2_angles/Chinese14.csv"
with open(filepath, 'r') as file:
    df = pd.read_csv(file)
print(euclidean_distance(df.iloc[1], df.iloc[103]))

104.28500402434139


In [10]:
def get_poses(filepath):
    with open(filepath, 'r') as file:
        df = pd.read_csv(file)
    distances = calculate_distances(df)
    outliers = non_minimum_suppression(df, distances)
    df_filtered = df.iloc[outliers]
    # print("Original DataFrame:")
    # print(df)
    # print("\nDataFrame after non-minimum suppression and dropping rows with distances exceeding the threshold:")
    # print(df_filtered)
    # print(outliers)
    return outliers, df_filtered

# filepath = "../data/2_angles/Chinese12.csv"
# frames, poses = get_poses(filepath)
# print(frames)

In [57]:
in_dir = "../data/2_angles"
out_dir = "../data/3_poses"

print("[")
for filename in os.listdir(in_dir):
    filepath = os.path.join(in_dir, filename)
    # with open(filepath, 'r') as file:
    #     data = json.load(file)
    # print(calculate_angles(parts_candidates))
    frames, poses = get_poses(filepath)

# read_json(filepath)
    output_path = os.path.join(out_dir, filename)
    poses.to_csv(output_path)
    print(f"\t{[filename, frames]},")
print("]")

[
	['Chinese12.csv', [24, 136, 196, 231, 318, 450, 475, 516, 540, 584, 600, 615, 650, 777, 821, 841, 900, 955, 1035, 1066, 1115, 1215, 1372, 1391, 1487, 1589, 1661, 1719, 1833]],
	['Chinese14.csv', [28, 129, 186, 209, 236, 357, 414, 428, 463, 489, 584, 792, 883, 989]],
	['Chinese15.csv', [7, 84, 223, 274, 285, 301, 364, 380, 421, 470, 545, 556, 622, 703, 804, 1004, 1106, 1130, 1198, 1333, 1385, 1441, 1513, 1659, 1686, 1714, 1729, 1739, 1774, 1840]],
	['Chinese16.csv', [12, 162]],
	['Chinese17.csv', [21, 144, 259, 337, 487, 515, 673, 753, 899]],
	['Chinese19.csv', [10, 519, 905, 1245, 1280, 1287, 1524, 1581, 2139, 2205]],
	['Chinese20.csv', [612, 1007, 1054, 1073, 1110, 1277, 1559, 1586, 1641, 1906, 2035]],
	['Indian1.csv', [3, 56, 115, 204, 302, 351, 360, 394, 476, 557, 669, 712, 747, 836, 885, 928, 961, 982, 1001, 1055, 1084, 1140, 1303, 1390, 1483, 1578, 1620, 1646, 1657, 1701, 1767, 1785, 1807, 1854, 1868, 1894, 1962, 1980, 1995, 2021, 2087, 2111, 2157, 2193, 2225, 2271, 2301, 2326,

In [11]:
import os
filename = "NeuralGlideMedianBias_train.csv"
filepath = os.path.join(in_dir, filename)
# with open(filepath, 'r') as file:
#     data = json.load(file)
# print(calculate_angles(parts_candidates))
frames, poses = get_poses(filepath)

# read_json(filepath)
output_path = os.path.join(out_dir, filename)
poses.to_csv(output_path)
print(f"\t{[filename, frames]},")

	['NeuralGlideMedianBias_train.csv', [38, 141, 205, 268, 352, 435, 576]],
